In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
abbreviation_to_name = {
    "AK": "Alaska",
    "AL": "Alabama",
    "AR": "Arkansas",
    "AZ": "Arizona",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "IA": "Iowa",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "MA": "Massachusetts",
    "MD": "Maryland",
    "ME": "Maine",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MO": "Missouri",
    "MS": "Mississippi",
    "MT": "Montana",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "NE": "Nebraska",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NV": "Nevada",
    "NY": "New York",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VA": "Virginia",
    "VT": "Vermont",
    "WA": "Washington",
    "WI": "Wisconsin",
    "WV": "West Virginia",
    "WY": "Wyoming",
    "DC": "District of Columbia",
    "ME-01": "ME-1",
    "ME-02": "ME-2",
    "NE-01": "NE-1",
    "NE-02": "NE-2",
    "NE-03": "NE-3"
}
name_to_abbreviation = {v: k for k, v in abbreviation_to_name.items()}

In [3]:
fte = pd.read_csv("538_daily_summary_2908.csv")
fte = fte[fte["variable"] == "state vote"]
fte["value"] = fte["value"].str.replace("%", "").astype("float")
pd.to_datetime(fte["model_date"], dayfirst="True")
fte.head()

,model_date,timestamp,state_abb,party,variable,metric,value
168,28/08/2024,2024-08-28T16:32:11Z,WY,REP,state vote,p_win,0.9999
169,28/08/2024,2024-08-28T16:32:11Z,WY,DEM,state vote,p_win,0.0001
170,28/08/2024,2024-08-28T16:32:11Z,WV,REP,state vote,p_win,1.0000
171,28/08/2024,2024-08-28T16:32:11Z,WV,DEM,state vote,p_win,0.0000
172,28/08/2024,2024-08-28T16:32:11Z,WI,REP,state vote,p_win,0.3829


In [4]:
pivot = pd.pivot_table(fte[fte["metric"] == "median"], values="value", columns=["party"], index=["state_abb", "model_date"]).reset_index()
pivot.set_index("state_abb", inplace=True)
pivot.columns.name = ""
pivot["model"] = "538"
pivot.head()

,model_date,DEM,IND,REP,model
state_abb,,,,,
AK,01/08/2024,0.4061,0.0506,0.5338,538
AK,02/08/2024,0.4072,0.0493,0.5334,538
AK,03/08/2024,0.4075,0.0495,0.5330,538
AK,04/08/2024,0.4083,0.0496,0.5331,538
AK,05/08/2024,0.4098,0.0488,0.5324,538


In [5]:
piv2 = pd.pivot_table(fte[fte["metric"] == "p_win"], values="value", columns=["party"], index=["state_abb", "model_date"]).reset_index()
piv2.set_index("state_abb", inplace=True)
piv2.columns.name = ""
piv2["model"] = "538"
piv2.rename(columns={"DEM":"DEM_win_prob", "REP":"REP_win_prob", "IND":"IND_win_prob"}, inplace=True)
piv2.head()

,model_date,DEM_win_prob,IND_win_prob,REP_win_prob,model
state_abb,,,,,
AK,01/08/2024,0.0979,0.0,0.9021,538
AK,02/08/2024,0.0977,0.0,0.9023,538
AK,03/08/2024,0.0988,0.0,0.9012,538
AK,04/08/2024,0.0996,0.0,0.9004,538
AK,05/08/2024,0.1057,0.0,0.8943,538


In [16]:
fte = pd.merge(pivot, piv2, on=["state_abb", "model_date", "model"])
fte["model_date"] = pd.to_datetime(fte["model_date"], format="mixed")
fte.reset_index(inplace=True)
fte.reset_index(drop=True, inplace=True)
fte.head()

,state_abb,model_date,DEM,IND,REP,model,DEM_win_prob,IND_win_prob,REP_win_prob
0,AK,2024-01-08,0.4061,0.0506,0.5338,538,0.0979,0.0,0.9021
1,AK,2024-02-08,0.4072,0.0493,0.5334,538,0.0977,0.0,0.9023
2,AK,2024-03-08,0.4075,0.0495,0.5330,538,0.0988,0.0,0.9012
3,AK,2024-04-08,0.4083,0.0496,0.5331,538,0.0996,0.0,0.9004
4,AK,2024-05-08,0.4098,0.0488,0.5324,538,0.1057,0.0,0.8943


In [15]:
s = pd.read_csv("silver_daily_summary_2908.csv")
s = s[["modeldate", "state", "winstate_inc", "winstate_chal", "winstate_3rd", "voteshare_inc", "voteshare_chal", "voteshare_3rd"]]
s["modeldate"] = pd.to_datetime(s["modeldate"], format="mixed")
s.rename(columns={"modeldate":"model_date", "state":"state_abb", "winstate_inc":"DEM_win_prob", "winstate_chal":"REP_win_prob", "winstate_3rd":"IND_win_prob", "voteshare_inc":"DEM", "voteshare_chal":"REP", "voteshare_3rd":"IND"}, inplace=True)
s["state_abb"] = s["state_abb"].map(name_to_abbreviation)
s["model"] = "Silver Bulletin"
s.reset_index(drop=True, inplace=True)
s.head()

,model_date,state_abb,DEM_win_prob,REP_win_prob,IND_win_prob,DEM,REP,IND,model
0,2024-07-29,AK,0.162750,0.824450,0.012800,39.092072,53.119499,5.483664,Silver Bulletin
1,2024-07-29,RI,0.947500,0.050700,0.001800,54.859692,39.466190,4.402967,Silver Bulletin
2,2024-07-29,OK,0.001100,0.997425,0.001475,29.431549,65.187393,4.012353,Silver Bulletin
3,2024-07-29,OH,0.069225,0.930100,0.000675,42.158699,53.210270,3.659906,Silver Bulletin
4,2024-07-29,MI,0.419650,0.580225,0.000125,46.830410,48.223419,3.914810,Silver Bulletin


In [14]:
ns = pd.read_csv("NS_daily_summary_2908.csv")
ns.rename(columns={"Unnamed: 0":"state_abb", "Harris":"DEM", "Trump":"REP", "Harris win prob":"DEM_win_prob", "Trump win prob":"REP_win_prob"}, inplace=True)
ns["state_abb"] = ns["state_abb"].map(name_to_abbreviation)
ns.drop(columns=["Winner", "Margin", "EC Votes"], inplace=True)
ns["DEM_win_prob"] = ns["DEM_win_prob"].str.replace("%", "").astype("int") / 100
ns["REP_win_prob"] = ns["REP_win_prob"].str.replace("%", "").astype("int") / 100
ns["model_date"] = datetime.datetime(year=2024, month=8, day=29)
ns["model_date"] = pd.to_datetime(ns["model_date"], format="ns")
ns["model"] = "New Statesman"
ns["IND"] = 0
ns["IND_win_prob"] = 0
ns.reset_index(drop=True, inplace=True)
ns.head()

,state_abb,DEM,REP,DEM_win_prob,REP_win_prob,model_date,model,IND,IND_win_prob
0,AK,40.3,51.4,0.03,0.97,2024-08-29,New Statesman,0,0
1,HI,61.2,32.9,1.00,0.00,2024-08-29,New Statesman,0,0
2,WA,55.5,37.4,1.00,0.00,2024-08-29,New Statesman,0,0
3,OR,54.0,39.0,0.99,0.01,2024-08-29,New Statesman,0,0
4,CA,61.0,32.9,1.00,0.00,2024-08-29,New Statesman,0,0


In [13]:
e = pd.read_csv("economist_daily_summary_2908.csv")
e.rename(columns={"Unnamed: 0":"state_abb", "Harris":"DEM", "Trump":"REP", "Harris win prob":"DEM_win_prob", "Trump win prob":"REP_win_prob"}, inplace=True)
e["state_abb"] = e["state_abb"].map(name_to_abbreviation)
e.drop(columns=["Winner", "Margin", "EC Votes"], inplace=True)
e["DEM_win_prob"] = e["DEM_win_prob"].str.replace("%", "").astype("int") / 100
e["REP_win_prob"] = e["REP_win_prob"].str.replace("%", "").astype("int") / 100
e["model_date"] = datetime.datetime(year=2024, month=8, day=29)
e["model_date"] = pd.to_datetime(e["model_date"], format="ns")
e["model"] = "Economist"
e["IND"] = 0
e["IND_win_prob"] = 0
e.reset_index(drop=True, inplace=True)
e.head()

,state_abb,DEM,REP,DEM_win_prob,REP_win_prob,model_date,model,IND,IND_win_prob
0,AL,37.5,62.5,0.00,1.00,2024-08-29,Economist,0,0
1,AK,44.6,55.4,0.05,0.95,2024-08-29,Economist,0,0
2,AZ,50.2,49.8,0.45,0.55,2024-08-29,Economist,0,0
3,AR,36.2,63.8,0.00,1.00,2024-08-29,Economist,0,0
4,CA,63.9,36.1,1.00,0.00,2024-08-29,Economist,0,0


In [20]:
df = pd.concat([fte, s, ns, e], axis=0, ignore_index=True)
df.head()

,state_abb,model_date,DEM,IND,REP,model,DEM_win_prob,IND_win_prob,REP_win_prob
0,AK,2024-01-08,0.4061,0.0506,0.5338,538,0.0979,0.0,0.9021
1,AK,2024-02-08,0.4072,0.0493,0.5334,538,0.0977,0.0,0.9023
2,AK,2024-03-08,0.4075,0.0495,0.5330,538,0.0988,0.0,0.9012
3,AK,2024-04-08,0.4083,0.0496,0.5331,538,0.0996,0.0,0.9004
4,AK,2024-05-08,0.4098,0.0488,0.5324,538,0.1057,0.0,0.8943


In [21]:
df.to_csv("all_models.csv")